In [1]:
import sys
sys.path.insert(19, 'litepose-pose-estimation/src')
sys.path.insert(20, 'src/referee_gloves_detector')
import json
import torch
from torchvision import transforms
import lp_config.lp_common_config as cc
from lp_model.lp_litepose import LitePose
from lp_inference.lp_inference import inference, assocEmbedding
from lp_utils.lp_image_processing import drawHeatmap, drawKeypoints, drawSkeleton
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
from preprocessing import red_filtering, segmentation_and_cropping, equalizing, squaring
import math

In [2]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_big_arch = config_data['path_big_arch']
file_path_csv_keypoints_webcam = config_data['path_csv_keypoints_webcam']

In [3]:
model = LitePose().to(cc.config["device"])
model.load_state_dict(torch.load(file_path_big_arch, map_location=cc.config["device"]))

<All keys matched successfully>

In [4]:
#uri = "src/referee_gloves_detector/resources/example_shapes.png"
#uri = "src/referee_gloves_detector/resources/example_shapes_1.png"
#uri = "src/referee_gloves_detector/resources/example_shapes_2.png"
#uri = "src/referee_gloves_detector/resources/bloisi_nao_red_hands.png"
#uri = "src/referee_gloves_detector/resources/refeere_scaled.png"
#uri = "src/referee_gloves_detector/resources/person_with_red_boxing_gloves.png"
#uri = "src/referee_gloves_detector/resources/refeere_scaled_v2.png"
uri = "src/referee_gloves_detector/resources/bloisi_nao_red_hands_v2.png"

In [5]:
# RED FILTERING

image = cv2.imread(uri)
full_mask = red_filtering(image)
cv2.imshow('mask', full_mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# SEGMENTATION E CROPPING

cropped_image = segmentation_and_cropping(image, full_mask)
cv2.imshow('cropped', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
# NORMALIZATION

normalized_image = cv2.normalize(cropped_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
cv2.imshow('normalized', normalized_image)
cv2.imshow('cropped', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
# EQUALIZATION

equalized_image = equalizing(cropped_image)
cv2.imshow('cropped', cropped_image)
cv2.imshow('equalized', equalized_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# SQUARING IMAGE

image = cv2.imread(uri)
print(image.shape)
squared_image = squaring(image)
print(squared_image.shape)

(235, 156, 3)
(224, 156, 3)


In [10]:
# Apro la webcam e salvo al volo i frame e il timestamp

webcam=cv2.VideoCapture(0) 

if not webcam.isOpened():
    raise Exception("Errore nell'apertura della webcam")

keypoints_vec = []
timestamps = []
resize = transforms.Resize([224, 224])  
to_tensor = transforms.ToTensor()
ret,frame=webcam.read()
while ret:
    
    
    istante_attuale = datetime.now()
    stringa_istante = istante_attuale.strftime("%d/%m/%Y %H:%M:%S.%f")
    timestamps.append(stringa_istante) 
    
    frame = cv2.flip(frame, 1)
    #-----------------------------PREPROCESSING START HERE-----------------------------
    #RED FILTERING
    full_mask = red_filtering(frame)
    #SEGMENTATION E CROPPING
    cropped_image = segmentation_and_cropping(frame, full_mask)
    #NORMALIZATION
    normalized_image = cv2.normalize(cropped_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #EQUALIZATION
    equalized_image = equalizing(normalized_image)
    #------------------------------PREPROCESSING END HERE------------------------------
    img = cv2.cvtColor(equalized_image, cv2.COLOR_BGR2RGB)
    img = squaring(img)
    
    try:
        # Può dare problemi se gli arriva una immagine con una dimensione mancante,
        # non ho capito il perchè (anche perchè è una condizione difficilmente riproducibile) 
        # ma questo può succedere
        im_pil = Image.fromarray(img)
    except:
        print("Shape dell'immagine che fa crashare il processo: " + str(img.shape))
        pass
    frame = resize(im_pil)
    tensor = to_tensor(frame)
    tensor = tensor.unsqueeze(0)
    
    if ret==True:
        output, keypoints = inference(model, tensor)
        keypoints_vec.append(keypoints)
        embedding = assocEmbedding(keypoints)
        frame_modified = drawSkeleton(tensor[0], embedding[0])
        #frame = drawKeypoints(tensor[0], keypoints[0])
        cv2.imshow("Pose estimation", frame_modified)
        key=cv2.waitKey(1) & 0xFF
        if key==ord("q"):
            break         
    ret, frame = webcam.read()
webcam.release()
cv2.destroyAllWindows()
print("Dimensioni dell'array di frame:", len(keypoints_vec))
print("Dimensioni dell'array di timestamp:", len(timestamps))

Dimensioni dell'array di frame: 29
Dimensioni dell'array di timestamp: 29


In [ ]:
# Salvo solo i 9 keypoints scelti per ogni frame

d = {'timestamp':[],'x': [], 'y': [], 'tag':[]}
df_keypoints = pd.DataFrame(data=d)
for timestamp, restricted_keypoints in zip(timestamps, keypoints_vec):
    for i in range(len(restricted_keypoints[0])):
        if not (restricted_keypoints[0][i]):
            df_temp = pd.DataFrame([{'timestamp': '01/01/1970 01:00:00','x':np.nan, 'y':np.nan, 'tag':np.nan}])
            df_keypoints = pd.concat([df_keypoints, df_temp])
        else:
            df_temp = pd.DataFrame(restricted_keypoints[0][i])
            df_temp['timestamp'] = timestamp
            df_keypoints = pd.concat([df_keypoints, df_temp])
df_keypoints.to_csv(file_path_csv_keypoints_webcam, index=False)
df_multiindex = df_keypoints.set_index(['timestamp'])
display(df_multiindex)